# Домашнее задание № 7 

In [12]:
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor

import os
import cv2
import skimage
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import time
from sklearn.linear_model import RANSACRegressor
from scipy.spatial import distance
from scipy.spatial.distance import pdist
from sklearn.cluster import AgglomerativeClustering
import time
import random

# Поиск объектов в видеопотоке 

Задача - придумать и реализовать алгоритм поиска (обнаружения без классификации) движущихся объектов.

В качестве исходных данных приведена выборка с видеофайлами и аннотацией для каждого кадра файла. Аннотация задана в виде ограничивающих прямоугольников в формате ```(y1,x1,y2,x2)```, где
- ```(x1,y1)``` - верхний левый угол прямоугольника;
- ```(x2,y2)``` - нижний правый угол прямоугольника.

Ссылка на данные – https://disk.yandex.ru/d/RdjMDoQQO8Ngcw

В качестве обучающей можно брать любые видеофайлы. При этом должны быть отдельно выбраны тестовые данные, которые не будут использованы в создании решения. 

Видеофайл с результатами работы алгоритма должен быть прикреплен вместе с решением. Пример фрагмента видеофайла с результатом поиска объектов приведен ниже.

Исходный код может быть в формате ```.py``` или ```.ipynb```.

![annotation](annot_example.gif "annotation")

## Требования к результату
- поиск должен находить геометрические место объекта на видеоизображении. Геометрическое место задано ограничивающим прямоугольником (bounding box);
- продолжительность решения для любого одного видеофайла не должна превышать 10 минут;
- должна быть приведена оценка точности решения;
- привести демонстрацию результатов требется на одном из тестовых видеофайлов.

In [13]:
# вспомогательная функция
def plot_one_image(src_image, is_gray=False):
    """
    Отрисовать с помощью plt исходное изображение.
    
    :param src_image: np.ndarray: исходное изображение
    :param is_gray: bool: флаг для отображения ЧБ изображений
    :return: None
    """
    fig, m_axs = plt.subplots(1, 1, figsize=(6*2, 4*2), constrained_layout=True)
    ax1 = m_axs

    cmap = 'gray' if is_gray else None
    ax1.set_title('Исходное изображение')
    ax1.imshow(src_image, cmap=cmap)
    ax1.set_xticks([]), ax1.set_yticks([])
    plt.show()

In [14]:
# def draw_points(image, points, rect=0, gray=True):
#     color1 = (0, 255, 0)
#     color2 = (0, 0, 255)
#     output = image.copy()
#     if gray:
#         output = cv2.cvtColor(image.copy(), cv2.COLOR_GRAY2RGB)
        
#     for point in points:
#         output = cv2.circle(output, point, 3, color1, -1)
    
#     if type(rect) != int:
#         output = cv2.rectangle(output, rect[0], rect[1], color2)
    
#     plot_one_image(output)

In [15]:
def get_video_details(cap, do_print=True):
    cnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fps = cap.get(cv2.CAP_PROP_FPS)

    if do_print:
        print(cnt, w, h, fps)
    return cnt, w, h, fps

In [16]:
def get_video(num=-1, file_name="", path='./Videos/Videos/'):
    """
    Функция достающая видео
    num - номер видео (если num == -1, то достаются все видео)
    
    """
    if file_name != "":
        return cv2.VideoCapture(path+file_name)
    
    all_names = os.listdir(path)
    names = []
    for name in all_names:
        if name.endswith(".mov") or name.endswith(".mp4"):
            names.append(name)
            
    if num >= 0:
        print(name)
        name_vid = names[num]
        return cv2.VideoCapture(path+name_vid)
        
        
    videos = []
    for name_vid in names:
        videos.append(cv2.VideoCapture(path+name_vid))
    return videos
    

In [17]:
def create_video(save_dir, size, img_format='jpg', vido_format='avi'):
    out_name = Path(save_dir).parts[-1]

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(Path(save_dir) / Path(f'{out_name}.{vido_format}')),
                          fourcc, 20, tuple(size.astype(int)))

    for fname in tqdm(sorted(map(str, Path(save_dir).glob(f'*.{img_format}')))):
        imag = skimage.io.imread(fname)
        imag = cv2.cvtColor(imag, cv2.COLOR_RGB2BGR)
        out.write(imag)
    out.release()

In [18]:
def show_video(num=-1,path='./tmp/', sleep=0):
    cap = get_video(num if num >=0 else 0, path=path)
    get_video_details(cap)
    cv2.startWindowThread()
    while (cap.isOpened()):
        is_ok, frame = cap.read()
        if not is_ok:
            break
            
        if sleep > 0:
            time.sleep(sleep)

        cv2.imshow("sparse optical flow", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [50]:
class AnnotationRect:
    def __init__(self, i, point1=None, point2=None):
        self.i = i
        self.x1 = point1[0] if point1 != None else -1 
        self.y1 = point1[1] if point1 != None else -1 
        self.x2 = point2[0] if point2 != None else -1 
        self.y2 = point2[1] if point2 != None else -1 
        
    def get_w(self):
        return self.x2 - self.x1

    def get_h(self):
        return self.y2 - self.y1
    
    def get_point1(self):
        return np.array([self.x1, self.y1])
    
    def get_point2(self):
        return np.array([self.x2, self.y2])
    
    def get_center(self):
        return (self.get_point1() + self.get_point2())//2
    
    def set_point1(self, point):
        self.x1 = point[0] 
        self.y1 = point[1]  
        
    def set_point2(self, point):
        self.x2 = point[0] 
        self.y2 = point[1]
        
    def is_there(self):
        return not(self.x1 == -1)

In [117]:
class Annotation:
    def get_annotation(self, file_name, path='./Videos/Annotation/'):
        f = open(path+file_name, 'r')
        
        self.annotation = []
        for string in f:
            l = string.split()
            self.annotation.append(AnnotationRect(l[1]))
            if len(l) > 3:
                self.annotation[-1].set_point1([int(l[4][:-1]), int(l[3][1:-1])])
                self.annotation[-1].set_point2([int(l[6][:-2]), int(l[5][:-1])])

        return self.annotation
    
    def show_annotation(self, cap, sleep=0):
        color = (0, 255, 0)
        cv2.startWindowThread()

        i = 0
        while (cap.isOpened()):
            is_ok, frame = cap.read()
            if not is_ok:
                break

            if sleep > 0:
                time.sleep(sleep)

            hsv_frame = cv2.cvtColor(frame.copy(), cv2.COLOR_BGR2HSV)
            frame = cv2.rectangle(frame, self.annotation[i].get_point1(), self.annotation[i].get_point2(), color)

            point1 = self.annotation[i].get_point1()
            point2 = self.annotation[i].get_point2()
            sl = hsv_frame[point1[1]:point2[1]+1, point1[0]:point2[0]+1]
            print(sl)
            hist = cv2.calcHist([sl[:,:,0]], [0], None, [180], [0, 180])
            plt.plot(hist)
            plt.show()

            cv2.imshow("sparse optical flow", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            i += 1

        cap.release()
        cv2.destroyAllWindows()
        
    def get_target_colors(self, name_file_vid):
        cap = get_video(file_name=name_file_vid)
        cnt, w, h, fps = get_video_details(cap)
        cnt = int(cnt)
        
        i = 0
        colors = []
        for idx in tqdm(range(cnt-1)):
            if not cap.isOpened():
                break
            is_ok, frame = cap.read()
            if not is_ok:
                break

            hsv_frame = cv2.cvtColor(frame.copy(), cv2.COLOR_BGR2HSV)

            point1 = self.annotation[idx].get_point1()
            point2 = self.annotation[idx].get_point2()
            sl = hsv_frame[point1[1]:point2[1]+1, point1[0]:point2[0]+1]
#             print('--------------')
#             plot_one_image(sl)
#             print(sl[:, 0])
#             print(sl[:, 1])
#             print(sl[:, 2])
            sl = np.reshape(sl, (-1, 3))
            
            if type(colors) == list:
                colors = sl.copy()
            else:
                colors = np.r_[colors, sl]

        cap.release()
        temp = np.ascontiguousarray(colors).view(np.dtype((np.void, colors.dtype.itemsize * colors.shape[1])))
        _, idx = np.unique(temp, return_index=True)
        colors = colors[idx]
        return colors

In [118]:
# annotation = Annotation()
# annotation.get_annotation('Clip_37_gt.txt')
# colors = annotation.get_target_colors('Clip_37.mov')
# colors

In [66]:
def get_transform_point(M, point):
    v = np.array([[point[0]], [point[1]], [1]])
    trans = M@v
    new_x = np.sum(trans[0])/np.sum(trans[2])
    new_y = np.sum(trans[1])/np.sum(trans[2])
    return np.array([new_x, new_y])

In [67]:
class Trajectory:
    def __init__(self, idx):
        self.points = []
        self.d = np.array([0,0], dtype=np.float32)
        self.idx = idx
    
    def add_new_step(self):
        self.points.append([])
    
    def add_point(self, point):
        if type(self.points[-1]) == list:
            self.points[-1] = np.array(point)
        else:
            self.points[-1] = np.r_[self.points[-1], np.array(point)]
        
    def add_points(self, points):
        if type(self.points[-1]) == list:
            self.points[-1] = np.array(points)
        else:
            self.points[-1] = np.r_[self.points[-1], np.array(points)]
    
    def get_last(self):
        return np.array(self.points[-1])
    
    def get_trajectory_len(self):
        return len(self.points)
    
    def add_d(self, v_move):
        self.d += v_move
        
    def get_d(self):
        return np.sum(self.d**2)
    
    def get_points(self, idx):
        if idx < self.idx or self.idx+self.get_trajectory_len()-1 < idx:
            return [np.array([-1,-1])]
        return self.points[idx-self.idx]
    
    def get_start_idx(self):
        return self.idx
    
    def get_end_idx(self):
        return self.idx+self.get_trajectory_len()-1
    
    def get_min_dist(self, points, M, return_vector = False):
        # Использовать после add_new_step
        
        f_points1 = self.points[-2].astype(np.float32)
        f_points1 = np.array([get_transform_point(M, point) for point in f_points1])
        
        f_points2 = points.astype(np.float32)
        
        dists = distance.cdist(f_points1, f_points2, 'euclidean')
        
        return np.min(dists)

    
    def get_av_diameter(self):
        if type(self.points[-1]) == list:
            self.points.pop()
        diams = np.array([np.max(distance.pdist(points_idx, 'euclidean')) for points_idx in self.points])
        return diams.mean()

In [68]:
class Trajectories:
    def __init__(self):
        self.trajectories = []
        self.save_trajectories = []
        self.calc_all_tr = False
        
    def get_last_points(self):
        temp_list = self.trajectories
        if self.calc_all_tr:
            temp_list = self.trajectories + self.save_trajectories
        return np.float32([np.array([trajectory.get_last()]) for trajectory in temp_list])
    
    
    def continue_trajectories(self, new_points, old_points, idx, M):
        t1 = time.time()
        count_trajectories = len(self.trajectories)
        clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=15, linkage='single').fit(old_points)
        labels = clustering.labels_
        t2 = time.time()
        if count_trajectories == 0:
            for l in np.unique(labels):
                self.trajectories.append(Trajectory(idx))
                self.trajectories[-1].add_new_step()
                inds = np.where(labels == l)[0]
                self.trajectories[-1].add_points(new_points[inds])
                return
            
        for trajectory in self.trajectories:
            trajectory.add_new_step()
        
        calc_ind = []
        last_i = len(old_points)
        for l in np.unique(labels):
            inds = np.where(labels == l)[0]
            dists = [trajectory.get_min_dist(old_points[inds], M) for trajectory in self.trajectories[:count_trajectories]]
            ind = np.argmin(dists)
            if dists[ind] < 5:
                self.trajectories[ind].add_points(new_points[inds])
                
                dists = distance.cdist(old_points[inds], new_points[inds], 'euclidean')
                p = np.where(dists == np.min(dists))
                ind1 = p[0][0]
                ind2 = p[1][0]
                v_move = new_points[ind2] - old_points[ind1]
                self.trajectories[ind].add_d(v_move)
                calc_ind.append(ind)
            else:
                self.trajectories.append(Trajectory(idx))
                self.trajectories[-1].add_new_step()
                self.trajectories[-1].add_points(new_points[inds])
        t3 = time.time()
#         print("t1", t2-t1)
#         print("t2", t3-t2)
#         print('count l', len(np.unique(labels)))
        
        
        
        calc_ind = set(calc_ind)
#         print('calc_ind', len(calc_ind))
#         print('count_trajectories', count_trajectories)
#         print('new_count_trajectories', len(self.trajectories))
        
        all_ind = set(range(count_trajectories))
        del_ind = list(set(calc_ind) ^ set(all_ind))
        del_ind = sorted(del_ind, reverse=True)
        for ind in del_ind:
            self.save_trajectories.append(self.trajectories[ind])
            self.trajectories.pop(ind)
    
    def get_trajectories_len(self):
        temp_list = self.trajectories
        if self.calc_all_tr:
            temp_list = self.trajectories + self.save_trajectories
        return np.array([trajectory.get_trajectory_len() for trajectory in temp_list])
    
    def get_trajectories_dist(self):
        temp_list = self.trajectories
        if self.calc_all_tr:
            temp_list = self.trajectories + self.save_trajectories
        return np.array([trajectory.get_d() for trajectory in temp_list])
          
    def get_points(self, idx):
        temp_list = self.trajectories
        if self.calc_all_tr:
            temp_list = self.trajectories + self.save_trajectories
        return np.array([trajectory.get_points(idx) for trajectory in temp_list])
    
    def load_all_trajectories(self):
        self.calc_all_tr = True
#         self.trajectories = self.trajectories + self.save_trajectories
    
    def del_save_trajectories(self):
        self.calc_all_tr = False
        
    def get_trajectories(self):
        temp_list = self.trajectories
        if self.calc_all_tr:
            temp_list = self.trajectories + self.save_trajectories
        return temp_list
    
    def get_trajectories_last_idx(self):
        temp_list = self.trajectories
        if self.calc_all_tr:
            temp_list = self.trajectories + self.save_trajectories
        return np.array([trajectory.get_end_idx() for trajectory in temp_list])
    
    def get_trajectories_av_diameter(self):
        temp_list = self.trajectories
        if self.calc_all_tr:
            temp_list = self.trajectories + self.save_trajectories
        return np.array([trajectory.get_av_diameter() for trajectory in temp_list])

In [69]:
def spec_points(image):
    """
    Функция для поиска особых точек и получения их дескрипторов 
    """
    
    hyp_params = dict(
        nfeatures = 200,
        nOctaveLayers = 10,
        contrastThreshold = 0.005,
        edgeThreshold = 6,
        sigma = 2.0)  # hyp params
    detector = cv2.SIFT_create(**hyp_params)

    keypoints, desc = detector.detectAndCompute(image.copy(), None)
    return keypoints, desc

In [84]:
def get_keyp_and_d(first, second):
    FLANN_INDEX_KDTREE = 2
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)

    flann = cv2.FlannBasedMatcher(index_params, search_params)
    
    ratio_thresh = 0.55
    keypoints1, desc1 = spec_points(first)
    keypoints2, desc2 = spec_points(second)
    
    matches = flann.knnMatch(desc1, desc2, k=2)
    
    good_matches = []
    for m, n in matches:
        if m.distance < ratio_thresh * n.distance:
            good_matches.append(m)
    
    return keypoints1, keypoints2, good_matches

In [85]:
def get_M(first, second):
    h, w = first.shape
    
    
    keypoints1, keypoints2, good_matches = get_keyp_and_d(first, second)
    
    pts1 = []
    pts2 = []   
    
    for good_matche in good_matches:
        pts1.append(keypoints1[good_matche.queryIdx].pt)
        pts2.append(keypoints2[good_matche.trainIdx].pt)
    
    pts1 = np.array(pts1).astype(np.float32)
    pts2 = np.array(pts2).astype(np.float32)
    
    model = RANSACRegressor()
    model.fit(pts1, pts2)

    points1 = np.array([[0,0], [0,h], [w,0], [w,h]]).astype(np.float32)
    points2 = model.predict(points1).astype(np.float32)
    M = cv2.getPerspectiveTransform(points1, points2)
    return M, model
    

In [86]:
def get_border_mask(gray):
    temp = cv2.GaussianBlur(gray, (5,5), 0)
#     temp2 = cv2.GaussianBlur(temp, (3,3), 0)
#     mask = np.uint8(np.abs(gray.astype(int)-temp.astype(int)) > 0)
#     kernel = np.ones((3, 3), np.uint8)
#     mask = cv2.dilate(mask,kernel,iterations = 1)
    kernel = np.ones((3, 3), np.uint8)
    mask = skimage.feature.canny(gray, sigma=1)
#     mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    return mask

In [87]:
def get_good_points(frame, gray, prev_gray, M, colors, k_size=3):
    h, w, _ = frame.shape
    hsv_frame = cv2.cvtColor(frame.copy(), cv2.COLOR_BGR2HSV)
    
    trans_prev_gray = cv2.warpPerspective(prev_gray.copy(), M, (prev_gray.shape[1], prev_gray.shape[0]))

        
    trans_gray = gray*(trans_prev_gray!=0).astype(np.uint8)
    nice_pic = np.abs(trans_prev_gray.astype(int)-trans_gray.astype(int)).astype(np.uint8)
    
    mask = np.uint8(get_border_mask(prev_gray))*255
    mask = cv2.warpPerspective(mask.copy(), M, (prev_gray.shape[1], prev_gray.shape[0]))
    mask2 = np.uint8(get_border_mask(gray))*255
    kernel = np.ones((3, 3), np.uint8)
#     mask2 = cv2.dilate(mask2,kernel,iterations = 1)
#     plot_one_image(mask, True)
#     plot_one_image(mask2, True)
    
    res_mask = np.uint8((mask.astype(np.int32) - mask2.astype(np.int32)) > 200)
#     plot_one_image(res_mask*255, True)
    
    p = np.where(res_mask > 0)
    y = p[0]
    x = p[1]
    all_points = np.array(list(zip(x, y)))
    
    points = np.array([all_points[0]])
    
    for point in all_points:
        if (point[1] >= h*20//40) and (point[1] <= h*28//40) and (point[0] <= w*15//90):
            continue
        
        if (point[1] > h-3) or (point[1] < 2) or (point[0] < 2) or (point[0] > w-3):
            continue
            
        if np.sum(trans_gray[point[1]-1:point[1]+2, point[0]-1:point[0]+2] == 0) > 1:
            continue
        
        if np.min(np.abs(points[:,0] - point[0]) + np.abs(points[:,1] - point[1])) > 10:
            col = hsv_frame[point[1], point[0]]
            if col[0] > 100 and np.any(np.prod(colors == col, axis=1).astype(bool)):
                points = np.r_[points, [point]]
    
    points = np.array([[np.float32(point)] for point in points])
    return points, trans_gray, trans_prev_gray

In [88]:
# cap = get_video(2)
# trajectories, list_frames = get_trajectory(cap, 100, True)
# # create_video('tmp', size=np.array((1920.0 , 1080.0)), vido_format='mp4')


In [89]:
def del_stick(list_trajectories, tr_dist, w, h):
    good_trajectories = []
    new_tr_dist = []
    for i, trajectory in enumerate(list_trajectories):
        points = trajectory.points
        
        fl = False
        for points_idx in points:
            if type(points_idx) == list:
                continue
            points_idx = np.array(points_idx)
            if np.any(points_idx[:, 0] > w//4) or np.any(points_idx[:, 1] < h//3):
                fl = True
                break
        if fl:
            good_trajectories.append(trajectory)
            new_tr_dist.append(tr_dist[i])
            
    return good_trajectories, np.array(new_tr_dist)

In [90]:
def get_av_diameter(self):
    if type(self.points[-1]) == list:
        self.points.pop()
        
#     print([len(distance.pdist(points_idx, 'euclidean')) for points_idx in self.points])
    diams = []
    for points_idx in self.points:
        if len(distance.pdist(points_idx, 'euclidean')) == 0:
            diams.append(0)
        else:
            diams.append(np.max(distance.pdist(points_idx, 'euclidean')))
    diams = np.array(diams)
    return diams.mean()

In [91]:
def draw_trajectories(list_image, trajectories):
    trajectories.load_all_trajectories()
    print(len(trajectories.trajectories))
    last_idx = len(list_image)-1
    tr_lens = trajectories.get_trajectories_len()
    tr_dist = trajectories.get_trajectories_dist()
    tr_end_idx = trajectories.get_trajectories_last_idx()
#     tr_av_diam = trajectories.get_trajectories_av_diameter()
    
    tr_av_diam = np.array([get_av_diameter(trajectory) for trajectory in trajectories.trajectories + trajectories.save_trajectories])

#     count, val = np.unique(tr_dist, return_counts=True)
#     plt.plot(val, count)
#     plt.show()
#     print(tr_dist)
    inds = (np.where(((tr_lens > 3) | (tr_end_idx == last_idx)) & (tr_dist/tr_lens > 2) & (tr_av_diam < 20))[0]).tolist()
    good_trajectories = [trajectory for i, trajectory in enumerate(trajectories.get_trajectories()) if i in inds]
    tr_dist = tr_dist[inds]
    
    h, w, _ = list_image[0].shape
#     good_trajectories, tr_dist = del_stick(good_trajectories, tr_dist, w, h)
    colors = [[random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)] for i in range(len(good_trajectories))]
    
    for idx in tqdm(range(len(list_image))):
        image = list_image[idx]
        points = [trajectory.get_points(idx) for trajectory in good_trajectories]
        
        good_inds = np.array([points[i][0][0] != -1 for i in range(len(points))])
        if np.all(good_inds == 0):
            cv2.imwrite(f'tmp2/frame_{str(idx).zfill(4)}.jpg', image.copy())
            continue
            
#         print(good_inds)
#         ind = np.where((tr_dist == np.min(tr_dist[good_inds])) & good_inds)[0][0]
#         print(ind)
        
        output = image.copy()
#         color = (0, 255, 0)
# #         print(points[ind])
#         for point in points[ind]:
#             output = cv2.circle(output, (point[0], point[1]), 3, color, -1)
            
        for i, tr_points in enumerate(points):
            if len(tr_points) == 0 or tr_points[0][0] == -1:
                continue
            for point in tr_points:
                output = cv2.circle(output, (point[0], point[1]), 3, colors[i], -1)
        cv2.imwrite(f'tmp2/frame_{str(idx).zfill(4)}.jpg', output)
    
    trajectories.del_save_trajectories()

In [108]:
# draw_trajectories(list_frames, trajectories)
# create_video('tmp2', size=np.array((1920.0 , 1080.0)), vido_format='mp4')
# show_video(path='./tmp2/', sleep=0.1)

In [113]:
def get_trajectory(cap, colors, max_idx=-1, create_vid=True):
    cnt, w, h, fps = get_video_details(cap)
    cnt = int(cnt)
    color = (0, 255, 0)
    feature_params = dict(maxCorners=1000, qualityLevel=0.02, minDistance=2, blockSize=7)
    lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    
    is_ok, first_frame = cap.read()
#     return first_frame
    if not is_ok:
        print("ERR: Can't read")
        return
    
    prev_gray = cv2.cvtColor(first_frame.copy(), cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(first_frame)
    idx = 0
    trajectories = Trajectories()
    list_frames = []
    all_time = 0
    for idx in tqdm(range(cnt-1)):
        if not cap.isOpened():
            break
        if max_idx > 0 and idx > max_idx:
            break
        is_ok, frame = cap.read()
        if not is_ok:
            break
        
        list_frames.append(frame.copy())
        gray = cv2.cvtColor(frame.copy(), cv2.COLOR_BGR2GRAY)
        mask = np.zeros_like(frame)
        
        t1 = time.time()
        M, model = get_M(prev_gray.copy(), gray.copy())

#         return nice_pic, gray
        t2 = time.time()
        prev, trans_gray, trans_prev_gray = get_good_points(frame, gray, prev_gray, M, colors)
        
#         #*----------
#         output = frame.copy()
#         for point in prev:
#             pointi = np.int32(point[0])
#             output = cv2.circle(output, (pointi[0], pointi[1]), 3, color, -1)
#         cv2.imwrite(f'tmp/frame_{str(idx).zfill(4)}.jpg', output)
#         print(len(prev))
#         return prev
#         #*----------
        
        nextp, status, error = cv2.calcOpticalFlowPyrLK(trans_prev_gray, trans_gray, prev, None, **lk_params)
        t3 = time.time()
        good_old = prev[status == 1].astype(int)
        good_new = nextp[status == 1].astype(int)

        new_points = []
        old_points = []
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv2.line(mask, (a, b), (c, d), color, 2)
            frame = cv2.circle(frame, (a, b), 3, color, -1)
            new_points.append(np.array([a, b]))
            old_points.append(np.array([c, d]))
        
        new_points = np.array(new_points)
        old_points = np.array(old_points)
        t4 = time.time()
#         if len(new_points):
#             trajectories.continue_trajectories(new_points, old_points, idx, M)
        t5 = time.time()
        output = cv2.add(frame, mask)
        prev_gray = gray.copy()
 
        cv2.imwrite(f'tmp/frame_{str(idx).zfill(4)}.jpg', output)
        
        print("-----------------------------")
        print("idx", idx)
        print("t M         ", t2-t1)
        print("t Потоки    ", t3-t2)
        print("t хз        ", t4-t3)
        print("t обновление", t5-t4)
        print("all         ", t5-t1)
        all_time += t5-t1
    cap.release()
#     draw_trajectories(list_frames, trajectories)
#     return trajectories, list_frames, all_time
    return list_frames, all_time

In [122]:
cap = get_video(file_name = "Clip_10.mov")
trajectories, list_frames, all_time = get_trajectory(cap, colors.copy(), 400, True)
# create_video('tmp', size=np.array((1920.0 , 1080.0)), vido_format='mp4')


1800.0 1920.0 1080.0 29.97


  0%|          | 0/1799 [00:00<?, ?it/s]

-----------------------------
idx 0
t M          1.6683354377746582
t Потоки     2.212172031402588
t хз         0.0
t обновление 0.0
all          3.880507469177246
-----------------------------
idx 1
t M          1.3775756359100342
t Потоки     2.51259708404541
t хз         0.0010170936584472656
t обновление 0.0
all          3.8911898136138916
-----------------------------
idx 2
t M          1.6769416332244873
t Потоки     2.4176149368286133
t хз         0.0
t обновление 0.0
all          4.094556570053101
-----------------------------
idx 3
t M          1.3490407466888428
t Потоки     2.126682758331299
t хз         0.0
t обновление 0.0
all          3.4757235050201416
-----------------------------
idx 4
t M          1.3689281940460205
t Потоки     2.1579055786132812
t хз         0.0010249614715576172
t обновление 0.0
all          3.5278587341308594
-----------------------------
idx 5
t M          1.3070507049560547
t Потоки     2.350034475326538
t хз         0.0
t обновление 0.0
all    

-----------------------------
idx 49
t M          1.3117680549621582
t Потоки     1.9765126705169678
t хз         0.0
t обновление 0.0
all          3.288280725479126
-----------------------------
idx 50
t M          1.3459525108337402
t Потоки     1.941096305847168
t хз         0.0
t обновление 0.0
all          3.287048816680908
-----------------------------
idx 51
t M          1.3382666110992432
t Потоки     1.9882218837738037
t хз         0.0
t обновление 0.0
all          3.326488494873047
-----------------------------
idx 52
t M          1.3075628280639648
t Потоки     2.0094029903411865
t хз         0.0
t обновление 0.0
all          3.3169658184051514
-----------------------------
idx 53
t M          1.3356492519378662
t Потоки     2.1979892253875732
t хз         0.0
t обновление 0.0010256767272949219
all          3.5346641540527344
-----------------------------
idx 54
t M          1.3185982704162598
t Потоки     2.1391587257385254
t хз         0.0
t обновление 0.0
all          3.4

-----------------------------
idx 98
t M          1.3559906482696533
t Потоки     1.6477248668670654
t хз         0.0
t обновление 0.0
all          3.0037155151367188
-----------------------------
idx 99
t M          1.3854913711547852
t Потоки     1.7115118503570557
t хз         0.0010080337524414062
t обновление 0.0
all          3.0980112552642822
-----------------------------
idx 100
t M          1.3597419261932373
t Потоки     1.7092280387878418
t хз         0.0
t обновление 0.0
all          3.068969964981079
-----------------------------
idx 101
t M          2.2514634132385254
t Потоки     1.979827642440796
t хз         0.0
t обновление 0.0
all          4.231291055679321
-----------------------------
idx 102
t M          1.485041618347168
t Потоки     1.7874138355255127
t хз         0.0
t обновление 0.0
all          3.2724554538726807
-----------------------------
idx 103
t M          1.6637599468231201
t Потоки     1.6159112453460693
t хз         0.0
t обновление 0.0
all         

-----------------------------
idx 146
t M          1.7850074768066406
t Потоки     2.302178382873535
t хз         0.0
t обновление 0.0
all          4.087185859680176
-----------------------------
idx 147
t M          1.7888357639312744
t Потоки     2.4458961486816406
t хз         0.0
t обновление 0.0
all          4.234731912612915
-----------------------------
idx 148
t M          1.9277074337005615
t Потоки     2.351606845855713
t хз         0.0
t обновление 0.0
all          4.279314279556274
-----------------------------
idx 149
t M          2.081631898880005
t Потоки     3.0955381393432617
t хз         0.0009975433349609375
t обновление 0.0
all          5.1781675815582275
-----------------------------
idx 150
t M          2.0073139667510986
t Потоки     2.381599187850952
t хз         0.0
t обновление 0.0
all          4.388913154602051
-----------------------------
idx 151
t M          1.8318040370941162
t Потоки     2.3825626373291016
t хз         0.0
t обновление 0.0
all          4

KeyboardInterrupt: 

In [184]:
print(all_time)
print(all_time/309)

1708.1440114974976
5.527974147241093


In [185]:
create_video('tmp2', size=np.array((1920.0 , 1080.0)), vido_format='mp4')

  0%|          | 0/401 [00:00<?, ?it/s]

In [186]:
show_video(path='./tmp2/', sleep=0.1)

tmp2.mp4
401.0 1920.0 1080.0 20.0


In [ ]:
draw_trajectories(list_frames, trajectories)

In [450]:
tr_lens = trajectories.get_trajectories_len()
tr_dist = trajectories.get_trajectories_dist()
print('count trajectories', len(tr_lens))
tr_dist = tr_dist[tr_lens > 5]
print('count normal trajectories', len(tr_dist))
print('dists', np.unique(tr_dist))

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1903323218.py, line 5)

In [401]:
np.unique(tr_lens)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])

In [22]:
create_video('tmp', size=np.array((1920.0 , 1080.0)), vido_format='mp4')

  0%|          | 0/401 [00:00<?, ?it/s]

In [23]:
show_video(path='./tmp2/', sleep=0.1)

tmp2.mp4
101.0 1920.0 1080.0 20.0


In [11]:
show_video(0, './Videos/Videos/', 1)

['Clip_1.mov', 'Clip_10.mov', 'Clip_11.mov', 'Clip_2.mov', 'Clip_3.mov', 'Clip_37.mov', 'Clip_4.mov', 'Clip_5.mov', 'Clip_6.mov', 'Clip_7.mov', 'Clip_8.mov', 'Clip_9.mov']
./Videos/Videos/Clip_1.mov
309.0 1920.0 1080.0 29.97


KeyboardInterrupt: 

In [14]:
show_video(sleep=0.2)

['tmp.mp4']
./tmp/tmp.mp4
308.0 1920.0 1080.0 20.0


In [17]:
create_video('tmp', size=np.array((1920.0 , 1080.0)), vido_format='mov')
show_video(sleep=0.2)

  0%|          | 0/308 [00:00<?, ?it/s]

['tmp.mov']
./tmp/tmp.mov
308.0 1920.0 1080.0 20.0
